In [18]:
from lib.models import ResNet
import torch
from torch import nn
from tqdm import tqdm
class CNNLSTM(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = ResNet().cuda()
        self.lstm = nn.LSTM(3,32)
        self.fc1 = nn.Linear(32,3)
    def forward(self,x_2d):
        x = torch.Tensor().cuda()
        for t in range(x_2d.size(1)):
            x_i = self.resnet(x_2d[:,t,:,:])
            x = torch.cat([x,x_i.unsqueeze(0)])
        out,_ = self.lstm(x)
        x = self.fc1(out)        
        # return x.view(-1,3,3)
        return x

In [17]:
model = CNNLSTM().cuda()
params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)
# model = ResNet().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)
train_losses = []
dev_losses = []
from lib.datasets import Dataset2p0
from torch.utils.data import DataLoader
data_dir = 'w3_small_balanced_normalized'
trainloader = DataLoader(Dataset2p0(dir=f'{data_dir}/train/',labels=f'{data_dir}/y_train.pt'),batch_size=64,shuffle=True)
devloader = DataLoader(Dataset2p0(dir=f'{data_dir}/test/',labels=f'{data_dir}/y_test.pt'),batch_size=64,shuffle=True)

Params:  10534


In [20]:
for (X,y) in tqdm(trainloader):
    logits = model(X.view(-1,3,1,5000).cuda())

100%|██████████| 828/828 [00:26<00:00, 30.94it/s]
